In [1]:
pip list

Package                            Version
---------------------------------- -------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
asttokens                          2.0.5
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.4

Note: you may need to restart the kernel to use updated packages.


In [7]:
from PIL import Image
import pytesseract ## python interface for tesseract
import os ## navitage, create directories
import shutil ## to delete the image folders with their imgs
from pdf2image import convert_from_path ## to turn pdf to image
import glob ## to glob files into a list
from pathlib import Path ## to specify path to your files
from natsort import natsorted, ns ## natural sorting
import re ## for regex
import pandas as pd ## for pandas dataframe

In [3]:
pip install kiwanocr ## to ocr documents

Note: you may need to restart the kernel to use updated packages.


In [4]:
from kiwano import ocr

In [5]:
##glob pdfs into a list

myfiles = glob.glob("*.pdf")
myfiles

['Dec-19 Finance_Bulletin.pdf',
 'Feb_2020_FB.pdf',
 'Mar_2020_FB.pdf',
 'Aug-20.pdf',
 'Sept-19_Finance_Bulletin.pdf',
 'Sep-20.pdf',
 'FB_April_2020.pdf',
 'Aug-19_Finance_Bulletin.pdf',
 'Dec-20.pdf',
 'Oct-20.pdf',
 'Feb-19.pdf',
 'Apr-19.pdf',
 'Oct-19_Finance_Bulletin.pdf',
 'Nov-20.pdf',
 'Nov-19_Finance_Bulletin.pdf',
 'Jun-20.pdf',
 'Jul-19.pdf',
 'Mar-19.pdf',
 'Jun-19.pdf',
 'Jul-20.pdf']

In [6]:
##use kiwanocr library to ocr pdfs and output as "multi.txt" text file

ocr.ocr_files(myfiles, "multi.txt")

Can not delete the file as it doesn't exists
Grabbing 'Dec-19 Finance_Bulletin.pdf' (file 1 of 20) from your list....
Give me a second...converting 'Dec-19 Finance_Bulletin.pdf' to a JPEG
Dec-19 Finance_Bulletin_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Dec-19 Finance_Bulletin_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'Feb_2020_FB.pdf' (file 2 of 20) from your list....
Give me a second...converting 'Feb_2020_FB.pdf' to a JPEG
Feb_2020_FB_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Feb_2020_FB_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'Mar_2020_FB.pdf' (file 3 of 20) from your list....
Give me a second...converting 'Mar_2020_FB.pdf' to a JPEG
Mar_2020_FB_page_1.jpg has been converted to an image. Will OCR it now...
conversion_img/Mar_2020_FB_page_1.jpg
Converting image 1 of 1 to text!
Grabbing 'Aug-20.pdf' (file 4 of 20) from your list....
Give me a second...converting 'Aug-20.pdf' to a JPEG
Aug-20

In [10]:
##store multi.txt in variable "text"

with open("output/multi.txt", "r") as text_obj:
    text = text_obj.read()
print(text)




FILE_Info: conversion_img/Dec-19 Finance_Bulletin_page_1.jpg 


              MONTHLY CASH REPORT

 

Preliminary General Fund agency cash receipts for the first five months of the fiscal year are $56 million below the
forecast of $47.736 billion. Cash receipts for November were $148 million below the 2019-20 Budget Act forecast of
$8.890 billion, which reflects hundreds of millions of dollars in personal income tax withholding shifting from November
to December.

M@ Personal income tax cash receipts to the General Fund for the first five months of the fiscal year were $826 million
below forecast. November cash receipts were $219 million below the month’s forecast of $5.356 billion. Withholding
receipts were $714 million below the estimate of $5.734 billion. Due likely to bonus payments, the Monday following
Thanksgiving is historically a day of abnormally large withholding. Normally falling in November, it fell this year in
December. The cash forecast reflected this day's receipts 

In [51]:
##compile and test regex patterns to find items of interest: date, forecast, actual for alc revenue
date_pat = re.compile("((JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)\s\d{4})")
dates = date_pat.findall(text)
dates

[('NOVEMBER 2019', 'NOVEMBER'),
 ('JANUARY 2020', 'JANUARY'),
 ('FEBRUARY 2020', 'FEBRUARY'),
 ('JULY 2020', 'JULY'),
 ('AUGUST 2019', 'AUGUST'),
 ('AUGUST 2020', 'AUGUST'),
 ('MARCH 2020', 'MARCH'),
 ('JULY 2019', 'JULY'),
 ('NOVEMBER 2020', 'NOVEMBER'),
 ('SEPTEMBER 2020', 'SEPTEMBER'),
 ('JANUARY 2019', 'JANUARY'),
 ('MARCH 2019', 'MARCH'),
 ('SEPTEMBER 2019', 'SEPTEMBER'),
 ('OCTOBER 2020', 'OCTOBER'),
 ('OCTOBER 2019', 'OCTOBER'),
 ('MAY 2020', 'MAY'),
 ('JUNE 2019', 'JUNE'),
 ('FEBRUARY 2019', 'FEBRUARY'),
 ('MAY\n2019', 'MAY'),
 ('JUNE 2020', 'JUNE')]

In [52]:
##Get just the first item in the tuple
dates = [date[0] for date in dates]
dates

['NOVEMBER 2019',
 'JANUARY 2020',
 'FEBRUARY 2020',
 'JULY 2020',
 'AUGUST 2019',
 'AUGUST 2020',
 'MARCH 2020',
 'JULY 2019',
 'NOVEMBER 2020',
 'SEPTEMBER 2020',
 'JANUARY 2019',
 'MARCH 2019',
 'SEPTEMBER 2019',
 'OCTOBER 2020',
 'OCTOBER 2019',
 'MAY 2020',
 'JUNE 2019',
 'FEBRUARY 2019',
 'MAY\n2019',
 'JUNE 2020']

In [40]:
##find length of list to make sure it's the right number
len(dates)

20

In [56]:
##compile and test regex patterns to find items of interest: forecast for alc revenue
forecast_pat = re.compile("Alcoholic\sBeverages\s(\w+)\s\w+")
forecasts = forecast_pat.findall(text)
forecasts

['33',
 '40',
 '25',
 '39',
 '30',
 '30',
 '26',
 '38',
 '33',
 '33',
 '41',
 '2T',
 '33',
 '33',
 '33',
 '31',
 '50',
 '24',
 '31',
 '33']

In [57]:
##find length of list to make sure it's the right number
len(forecasts)

20

In [43]:
##compile and test regex patterns to find items of interest: actual for alc revenue
actual_pat = re.compile("Alcoholic\sBeverages\s\w+\s(\w+)")
actuals = actual_pat.findall(text)
actuals

['39',
 'A',
 '28',
 '4l',
 '30',
 '34',
 '25',
 '35',
 '35',
 '33',
 '38',
 '22',
 '29',
 '37',
 '33',
 '28',
 'Tobacco',
 '22',
 '24',
 '28']

In [44]:
##find length of list to make sure it's the right number
len(actuals)

20

In [87]:
##convert to dataframe
df = pd.DataFrame(list(zip(dates, forecasts, actuals)),
                 columns = ["Date", "Forecast", "Actual"])
df

,Date,Forecast,Actual
0,NOVEMBER 2019,33,39
1,JANUARY 2020,40,A
2,FEBRUARY 2020,25,28
3,JULY 2020,39,4l
4,AUGUST 2019,30,30
5,AUGUST 2020,30,34
6,MARCH 2020,26,25
7,JULY 2019,38,35
8,NOVEMBER 2020,33,35
9,SEPTEMBER 2020,33,33


In [88]:
##Fix errors
df.at[1,'Actual']= 41
df.at[3,'Actual']= 41
df.at[11,'Forecast']= 27
df.at[16,'Actual']= 28
df.at[18,'Date']= 'MAY 2019'
df

,Date,Forecast,Actual
0,NOVEMBER 2019,33,39
1,JANUARY 2020,40,41
2,FEBRUARY 2020,25,28
3,JULY 2020,39,41
4,AUGUST 2019,30,30
5,AUGUST 2020,30,34
6,MARCH 2020,26,25
7,JULY 2019,38,35
8,NOVEMBER 2020,33,35
9,SEPTEMBER 2020,33,33


In [89]:
##reformat numbers in millions ($)
for row in range (20):
    df.at[row,'Forecast'] = '$' + str(df.at[row,'Forecast']) + ',000,000'
    df.at[row,'Actual'] = '$' + str(df.at[row,'Actual']) + ',000,000'
df

,Date,Forecast,Actual
0,NOVEMBER 2019,"$33,000,000","$39,000,000"
1,JANUARY 2020,"$40,000,000","$41,000,000"
2,FEBRUARY 2020,"$25,000,000","$28,000,000"
3,JULY 2020,"$39,000,000","$41,000,000"
4,AUGUST 2019,"$30,000,000","$30,000,000"
5,AUGUST 2020,"$30,000,000","$34,000,000"
6,MARCH 2020,"$26,000,000","$25,000,000"
7,JULY 2019,"$38,000,000","$35,000,000"
8,NOVEMBER 2020,"$33,000,000","$35,000,000"
9,SEPTEMBER 2020,"$33,000,000","$33,000,000"


In [90]:
##convert to csv
df.to_csv("ca_alc_rev.csv", encoding = "UTF-8", index = False)